In [0]:
%run "/Workspace/Users/anirudhp@megnity.com/Healthcare/src/Explore and Clean"

In [0]:
first_camp_cleaned.printSchema()
second_camp_cleaned.printSchema()
third_camp_cleaned.printSchema()
patient_profiles_cleaned.printSchema()
health_camp_details_cleaned.printSchema()

1. There are 5 tables. See if there is any relationship between them. 
2. If there is a relationship, see if it makes sense to put the related tables together in whichever way it makes sense
3. If required create new calculated or derived field if needed
4. The end outcome could be unified table or tables

In [0]:
# display(first_camp_cleaned.limit(25))
# display(second_camp_cleaned.limit(25))
# display(third_camp_cleaned.limit(25))
# display(patient_profiles_cleaned.limit(25))
display(health_camp_details_cleaned.limit(25))

1. We can join first_camp on patient_id with patient_profiles

In [0]:
joined_firstcamp_profiles = first_camp_cleaned.join(patient_profiles_cleaned, on="patient_id", how="left")
display(joined_firstcamp_profiles)
print("Number of rows in first_camp_cleaned:", first_camp_cleaned.count())
print("Number of rows in patient_profiles:", patient_profiles.count())

Join second camp with patient profiles using a left join

In [0]:
joined_secondcamp_profiles = second_camp_cleaned.join(patient_profiles_cleaned, on="patient_id", how="left")
display(joined_secondcamp_profiles)
print("Number of rows in second_camp_cleaned:", second_camp_cleaned.count())
print("Number of rows in patient_profiles:", patient_profiles.count())

Join third camp with patient profiles using left join

In [0]:
joined_thirdcamp_profiles = third_camp_cleaned.join(patient_profiles_cleaned, on="patient_id", how="left")
display(joined_thirdcamp_profiles)
print("Number of rows in third_camp_cleaned:", third_camp_cleaned.count())
print("Number of rows in patient_profiles:", patient_profiles.count())

In [0]:
# Select Patient_ID and Health_Camp_ID from each DataFrame
df_first_select = first_camp_cleaned.select(col("Patient_ID"), col("Health_Camp_ID"))
df_second_select = second_camp_cleaned.select(col("Patient_ID"), col("Health_Camp_ID"))
df_third_select = third_camp_cleaned.select(col("Patient_ID"), col("Health_Camp_ID"))

# Union the DataFrames to combine all Patient_ID and Health_Camp_ID pairs
df_combined = df_first_select.union(df_second_select).union(df_third_select)

# Remove duplicates to ensure unique pairs
df_combined_clean = df_combined.filter(
    (col("Patient_ID").isNotNull()) &
    (col("Health_Camp_ID").isNotNull())
).dropDuplicates()

# Display the first 25 rows of the combined DataFrame
display(df_combined_clean.limit(25))

# Print schema and row count
df_combined_clean.printSchema()
print(f"Row count: {df_combined_clean.count()}")

In [0]:
# Step 2: Select and rename Health_Camp_ID columns
try:
    # Select only Patient_ID and Health_Camp_ID, renaming Health_Camp_ID for clarity
    df_first_selected = first_camp_cleaned.select(
        col("Patient_ID"),
        col("Health_Camp_ID").alias("First_Camp_Health_ID")
    )
    df_second_selected = second_camp_cleaned.select(
        col("Patient_ID"),
        col("Health_Camp_ID").alias("Second_Camp_Health_ID")
    )
    df_third_selected = third_camp_cleaned.select(
        col("Patient_ID"),
        col("Health_Camp_ID").alias("Third_Camp_Health_ID")
    )
except Exception as e:
    print(f"Error selecting columns: {e}")
    raise

# Step 3: Perform outer joins to combine all Patient_IDs
try:
    # Join the three DataFrames on Patient_ID using outer joins
    df_combined = df_first_selected.join(
        df_second_selected,
        "Patient_ID",
        "outer"
    ).join(
        df_third_selected,
        "Patient_ID",
        "outer"
    )
    print("DataFrames joined successfully.")
except Exception as e:
    print(f"Error joining DataFrames: {e}")
    raise

# Step 4: Display sample output (first 10 rows)
display(df_combined.limit(10))

# Step 5: Save the output table to DBFS
# try:
#     df_combined.write.csv(
#         "dbfs:/Volumes/healthcare/health_camps/cleaned/Combined_Health_Camp_IDs.csv",
#         header=True,
#         mode="overwrite"
#     )
#     print("Output table saved to dbfs:/Volumes/healthcare/health_camps/cleaned/Combined_Health_Camp_IDs.csv")

In [0]:
df_combined.count()

### This is a outer join of all camps tables with patient id, health_camp_ids, with patient profiles details

In [0]:
# Perform a left join of df_combined with patient_profiles on Patient_ID
df_final = df_combined.join(patient_profiles, "Patient_ID", "left")

# Display the first 25 rows of the final DataFrame
display(df_final.limit(25))

# Print schema and row count
df_final.printSchema()
print(f"Row count: {df_final.count()}")

Check who all are in both first camp and second table attended

In [0]:
filtered_df = df_final.filter(
    (df_final.First_Camp_Health_ID.isNotNull()) & 
    (df_final.Second_Camp_Health_ID.isNotNull())
)
display(filtered_df)

### Combine all patient details table with all the information we have on them from the camp tables

Combine donation and health_score from first_camp_cleaned with matching patient_ID in df_final

In [0]:
from pyspark.sql.functions import col

# Perform an outer join with df_final and first_camp_cleaned on Patient_ID and health_camp_id
df_joined = df_final.alias("df1").join(
    first_camp_cleaned.select(
        col("patient_id").alias("Patient_ID"),
        col("health_camp_id").alias("First_Camp_Health_ID"),
        "donation",
        "health_score"
    ).alias("df2"),
    on=[col("df1.Patient_ID") == col("df2.Patient_ID"), col("df1.First_Camp_Health_ID") == col("df2.First_Camp_Health_ID")],
    how="outer"
)

# Select only the required columns and rename health_score to first_camp_health_score
df_final_updated = df_joined.select(
    col("df1.*"),
    col("df2.donation"),
    col("df2.health_score").alias("first_camp_health_score")
)

# Display the resulting DataFrame
display(df_final_updated)

### Now add health score from the second table

In [0]:
from pyspark.sql.functions import col

# Perform an outer join with df_final_updated and second_camp_cleaned on Patient_ID and health_camp_id
df_joined_second = df_final_updated.alias("df1").join(
    second_camp_cleaned.select(
        col("patient_id").alias("Patient_ID"),
        col("health_camp_id").alias("Second_Camp_Health_ID"),
        col("health_score").alias("second_camp_health_score")
    ).alias("df2"),
    on=[col("df1.Patient_ID") == col("df2.Patient_ID"), col("df1.Second_Camp_Health_ID") == col("df2.Second_Camp_Health_ID")],
    how="outer"
)

# Select only the required columns
df_final_updated = df_joined_second.select(
    col("df1.*"),
    col("df2.second_camp_health_score")
)

# Display the resulting DataFrame
display(df_final_updated)

### Now combine the reamining information from the third table into the df_final_updated

In [0]:
# Perform an outer join with df_final_updated and third_camp_cleaned on Patient_ID and health_camp_id
df_joined_third = df_final_updated.alias("df1").join(
    third_camp_cleaned.select(
        col("patient_id").alias("Patient_ID"),
        col("health_camp_id").alias("Third_Camp_Health_ID"),
        col("number_of_stall_visited"),
        col("last_stall_visited_number")
    ).alias("df2"),
    on=[col("df1.Patient_ID") == col("df2.Patient_ID"), col("df1.Third_Camp_Health_ID") == col("df2.Third_Camp_Health_ID")],
    how="outer"
)

# Select only the required columns
df_final_updated = df_joined_third.select(
    col("df1.*"),
    col("df2.number_of_stall_visited").alias("number_of_stall_visited_in_third_camp"),
    col("df2.last_stall_visited_number").alias("last_stall_visited_number_in_third_camp")
)

# Display the resulting DataFrame
display(df_final_updated)

### Save all the tables including the newly outer joined table

In [0]:
first_camp_cleaned.write.mode("overwrite").saveAsTable("first_camp_cleaned")
second_camp_cleaned.write.mode("overwrite").saveAsTable("second_camp_cleaned")
third_camp_cleaned.write.mode("overwrite").saveAsTable("third_camp_cleaned")
patient_profiles_cleaned.write.mode("overwrite").saveAsTable("patient_profiles_cleaned")
health_camp_details_cleaned.write.mode("overwrite").saveAsTable("health_camp_details_cleaned")
df_final_updated.write.mode("overwrite").saveAsTable("df_final_updated")

In [0]:
%sql
SELECT * 
FROM df_final_updated
LIMIT 10

In [0]:
first_camp_cleaned.write.format("delta").mode("overwrite").save("/delta/healthcare/camp_data/first_camp_cleaned")
second_camp_cleaned.write.format("delta").mode("overwrite").save("/delta/healthcare/camp_data/second_camp_cleaned")
third_camp_cleaned.write.format("delta").mode("overwrite").save("/delta/healthcare/camp_data/third_camp_cleaned")
health_camp_details.write.format("delta").mode("overwrite").save("/delta/healthcare/camp_data/health_camp_details")
df_final_updated.write.format("delta").mode("overwrite").save("/delta/healthcare/camp_data/df_final_updated")